In [1]:
import os 
import requests 
from dotenv import load_dotenv
from bs4 import BeautifulSoup 
from IPython.display import Markdown, display
from openai import OpenAI


In [2]:

# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:142.0) Gecko/20100101 Firefox/142.0"
}

class Website:
    def __init__(self, url):
        self.url = url 
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "Not Title found"

        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose() 
        self.text = soup.body.get_text(separator="\n", strip=True)
        

In [4]:
system_prompt = """
 You are a assistant that analyzes the contents of a website \ 
 and provide a short summary, ignoring text that might be navigation related.
 Respond in Markdown.

"""

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += """
        \n The content of this website is a follows; \
        please provide a short summary of this website in markdown. \
        If it includes news or announcements, then summarize these too. \n\n
        
    """

    user_prompt += website.text 
    return user_prompt

In [6]:
# Constants

OLLAMA_API = "http://localhost:11434/v1"
MODEL = "llama3.2"

ollama_via_openai = OpenAI(base_url=OLLAMA_API, api_key="ollama")
# Create a messages list using the same format that we used for OpenAI

def messages_for(website):
    return  [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]





In [7]:

def sumarize(url):
    website = Website(url) 
 
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    return response.choices[0].message.content
    
    

In [8]:
def display_summary(url):
    summary = sumarize(url)
    display(Markdown(summary))
    

In [9]:
display_summary("https://news.google.com/topics/CAAqLAgKIiZDQkFTRmdvSkwyMHZNR1ptZHpWbUVnVndkQzFDVWhvQ1FsSW9BQVAB?hl=pt-BR&gl=BR&ceid=BR%3Apt-419")

Um dos post mais interessantes foi sobre o lançamento do novo Tecno Spark Slim 5G, que apresenta um design fino, uma bateria grande e uma tela de 144 Hz.

O Tecno Spark Slim foi anunciado como um modelo muito acessível da marca chinesa para o mercado brasileiro. Ele conta com uma tela AMOLED de alta resolução de 6,43 polegadas e teclas físicas de tipo Linear Switch. 

Segundo a informação do site, a bateria é de 5.160 mAh e tem carga rápida de 30W que permite acender em apenas 35 minutos, com duas carregativas no dia.

O Tecno 5G com capacidade 256GB é um novo modelo que chegará ao Brasil com uma boa preço, em contraste com outras opções que estão no mercado como o Redmi Note 14 Midnight Black.